## Load datasets

In [1]:
import pandas as pd

# files = {
#     'none': pd.read_csv('../deepseek-coder-6.7b-instruct/deepseek-coder-6.7b-instruct.csv'),
#     'cot' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_deepseek-coder-6.7b-instruct.csv'),
#     'ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_deepseek-coder-6.7b-instruct.csv'),
# }

gold_standard = pd.read_csv('../datafiles/lcquad_test_gold.csv')

In [2]:
#model_output = pd.read_csv('../datafiles/deepseek_output_21_01_2024.csv')
model_output = pd.read_csv('../datafiles/codellama_output_22_01_2024.csv')
model_output = pd.read_csv('../datafiles/deepseek_fs_response_corrected_23_01_2024.csv')

In [3]:
model_output = model_output.fillna('none')

## Preprocess

In [4]:
# String to python data structure
model_output['evaluated'] = model_output['output'].apply(eval)
gold_standard['evaluated'] = gold_standard['output'].apply(eval)

In [5]:
def preprocess(output):
    _result = set()
    if 'head' in output and 'results' in output:
        subset = set()
        for element in output['results']['bindings']:
            subset |= ({element[x]['value'] for x in element.keys()})
        _result = subset
    elif 'head' in output and 'boolean' in output:
        _result = output['boolean']
    return _result

In [6]:
def check_well_constructed(output):
    if 'output error' in output:
        return False
    return True

In [7]:
def type_of_consult(consult):
    result = 'NONE'
    if 'COUNT' in consult: result = 'COUNT'
    elif 'ASK' in consult: result = 'ASK'
    elif 'ORDER BY' in consult: result = 'ORDER_BY'
    elif 'SELECT' in consult: result = 'SELECT'
    return result

In [8]:
model_output['curated'] = model_output['evaluated'].apply(preprocess)

In [9]:
gold_standard['curated'] = gold_standard['evaluated'].apply(preprocess)

In [10]:
model_output['method'] = 'all'

### Code Execution Eval

In [11]:
import numpy as np
grouped = model_output.groupby('method')
ex_eval = pd.DataFrame()
for i, (method, group) in enumerate(grouped):
    comparison = group['curated'].values == gold_standard['curated'][:100].values
    print(f"Accuracy for {method}: {comparison.mean()}")
    comparison = pd.DataFrame(comparison)
    ex_eval = pd.concat([ex_eval, comparison])

Accuracy for all: 0.39


In [12]:
model_output['evaluation'] = ex_eval[0].to_list()
model_output['evaluation'] = model_output['evaluation'].astype(int)

In [14]:
model_output['type'] = gold_standard.sparql_query.apply(type_of_consult)[:100].to_list() #* 8

In [15]:
model_output[['method','evaluation','type']].groupby(['method', 'type']).describe().round(2)

evaluation                                     
                   count  mean   std  min  25%  50%  75%  max
method type                                                  
all    ASK          10.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
       COUNT        14.0  0.07  0.27  0.0  0.0  0.0  0.0  1.0
       SELECT       76.0  0.18  0.39  0.0  0.0  0.0  0.0  1.0

In [26]:
model_output['valid'] = model_output['output'].apply(check_well_constructed)

In [16]:
model_output[['method','valid']].groupby('method').mean()

KeyError: "['valid'] not in index"